In [12]:
import numpy as np
import control as ct

In [13]:
#Intializations
Ts = 1E-2 #[s]
run_time = 30 #[s]

#Variables
g = 9.81 #gravity [m/s^2]
m = 1 #pendulum mass [kg]
l = 0.5 #pendulum length [m]
k = 0.5 #coeff static friction

#EKF Variables
var_Q = 1e-6 #process noise
var_R = 1e-6 #measurement noise
numb_sensor = 2 # TODO
Q1 = var_Q*np.eye(3) # TODO
Q2 = var_Q*np.eye(2) #TODO
P1 = var_R*np.eye(3) # TODO

R = var_R * np.eye(2*numb_sensor) # TODO
R2 = var_R * np.eye(2)  # TODO

#MFC EKF
"""
Add documentation about this guy 
"""
A_EKF = np.array([[1, Ts, 0.5*Ts**2],
                  [0, 1, Ts],
                  [0, 0, 1]])

beta = 1/(m*l**2) # From dynamics

B_EKF = np.array([[0.5*Ts**2*beta],
                  [Ts*beta],
                  0])
C_EKF = np.array([[1, 0, 0]])



set_point = np.pi/4
step_start = (10/Ts)

t = np.linspace(0,run_time,int(run_time/Ts))

states = np.zeros((3,1))
outputs = np.zeros((3,1))
u = np.zeros((1,1))
u_mfc = np.zeros((1,1))
int_value = 0

#Others???
x_hat = np.zeros((3,1))
sigmaV = 0
P_hat = P1 #Why does he call it P1?
prev_diff = 0

#Control Variables
e = 0
e_dot = 0
u_i = 0



C:\Users\zenwh\AppData\Local\Temp\ipykernel_10688\2342922792.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  B_EKF = np.array([[0.5*Ts**2*beta],


In [14]:
#Pole Placement
def pole_placement():
    #State Space
    A = np.array([[0, 1, 0],\
              [0, 0, 0],\
              [1, 0, 0]])

    B = np.array([[0],\
              [1],\
              [0]])

    C = np.array([[1, 0]])  

    #Convergence Specs
    Mp = 0.10
    Ts = 1.0
    v = np.log(Mp)
    zeta = -v/np.sqrt(np.pi**2 + v**2)
    Wn = 4/(zeta*Ts)

    #Poles
    P2 = np.roots([1, 2*zeta*Wn, Wn**2])
    P = np.array([P2[0], P2[1], -10*np.real(P2[0])])

    K = ct.acker(A,B,P)
    K1 = K[0,0]
    K2 = K[0,1]
    K3 = K[0,2]

    return K1,K2,K3

In [15]:
#Reference Generation
def ref_gen(step_start, int_value, final_value):
    ref = np.ones((1,int(step_start)))*int_value
    tmp = np.size(ref)
    end_t = int(run_time/Ts)
    ref = np.append(ref, np.ones((1, end_t-tmp)) * final_value)
    return ref

In [16]:
def non_linear_dynamics(in_states, u, ref):
    states[0,0] = in_states[1,0] #x1
    #states[1,0] = (g/l)*np.sin(in_states[0,0]) -(k/m)*in_states[1,0] + 1/(m*l**2)*u #x2
    #states[2,0] = ref - in_states[0,0] # z3?

    #states_out = (states*Ts) + in_states
    return  states

In [17]:
ref = ref_gen(step_start, int_value, set_point)
ref=np.reshape(ref, ((1,3000)))

print(np.shape(ref))
print(np.shape(u_mfc))
print(np.shape(outputs))

for k in range(2, np.size(t)):
    
    (states[:,k]) = non_linear_dynamics(outputs[:,k-1], u_mfc[:,k-1],ref[:,k])


(1, 3000)
(1, 1)
(3, 1)


IndexError: index 1 is out of bounds for axis 1 with size 1